# **Prueba Técnica LuloBank**
*Vanessa Movilla*

In [ ]:
!pip install ydata-profiling

Se importan las librerias necesarias

In [ ]:
# Librerias necesarias
import os
import glob
import json
import sqlite3
import requests
import pandas as pd
from datetime import date, timedelta
from ydata_profiling import ProfileReport

1. Obtener información del siguiente API Rest http://api.tvmaze.com trayendo todas las
series que se emitieron en enero del 2024.
Ayuda: para obtener las series emitidas el 1 de enero del 2024 se utilizó el siguiente
llamado http://api.tvmaze.com/schedule/web?date=2024-01-01
2. Almacenar los datos crudos (json).

In [ ]:
# Function to request the data from the API and store it in json files
def data_request(link,params, filename):
    result = requests.get(link,params)
    result.status_code
    if result.status_code == 200:
        data = result.json()
        print(f"\n Data for {filename} is retrieved successfully. \n")

        # Store the data in a file
        with open(filename, 'w') as f:
            json.dump(data, f, indent=4)  # Use indent for pretty printing

    else:
        print(f"\n ERROR retrieving data for {filename}")
        return None  # Indicate failure
    return data

# Create directory to store JSON files if it doesn't exist
if not os.path.exists('json'):
    os.makedirs('json')

# Loop through the days of January 2024
start_date = date(2024, 1, 1)
url ='http://api.tvmaze.com/schedule/web'

for i in range(31):
  date_data= start_date + timedelta(i)
  filename = os.path.join('json', f'tvmaze_data_{date_data}.json')
  data = data_request(url,{'date':date_data}, filename)


 Data for json/tvmaze_data_2024-01-01.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-02.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-03.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-04.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-05.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-06.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-07.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-08.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-09.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-10.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-11.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-12.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-13.json is retrieved successfully. 


 Data for json/tvmaze_data_2024-01-14.json is retr

3. Con base a los Json obtenidos del API, generar dataframe(s) (con la librería de su
elección, ej pandas, dask, polars) que conserve la integridad de los datos del Json.


In [ ]:
def json_to_dataframe(directory):

    all_data = []

    for filename in glob.glob(os.path.join(directory, '*.json')):
        try:
            with open(filename, 'r') as f:
                print(f"Processing {filename}")
                data = json.load(f)
                all_data.extend(data)

        except json.JSONDecodeError as e:
            print(f"Error decoding JSON in {filename}: {e}")
            return None  # Indicate failure
        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}")
            return None

    try:
        df = pd.json_normalize(all_data)
        return df
    except Exception as e:
        print(f"Error creating DataFrame: {e}")
        return None

df = json_to_dataframe('json')

if df is not None:
    print(df.head())

Processing json/tvmaze_data_2024-01-27.json
Processing json/tvmaze_data_2024-01-23.json
Processing json/tvmaze_data_2024-01-13.json
Processing json/tvmaze_data_2024-01-29.json
Processing json/tvmaze_data_2024-01-26.json
Processing json/tvmaze_data_2024-01-28.json
Processing json/tvmaze_data_2024-01-05.json
Processing json/tvmaze_data_2024-01-19.json
Processing json/tvmaze_data_2024-01-10.json
Processing json/tvmaze_data_2024-01-18.json
Processing json/tvmaze_data_2024-01-11.json
Processing json/tvmaze_data_2024-01-20.json
Processing json/tvmaze_data_2024-01-06.json
Processing json/tvmaze_data_2024-01-22.json
Processing json/tvmaze_data_2024-01-01.json
Processing json/tvmaze_data_2024-01-25.json
Processing json/tvmaze_data_2024-01-07.json
Processing json/tvmaze_data_2024-01-03.json
Processing json/tvmaze_data_2024-01-09.json
Processing json/tvmaze_data_2024-01-02.json
Processing json/tvmaze_data_2024-01-16.json
Processing json/tvmaze_data_2024-01-24.json
Processing json/tvmaze_data_2024

4. Realizar profiling a los datos y realizar un análisis.
Se realiza el profiling de los datos sin la columna de URL para facilitar la generación del reporte, teniendo en cuenta que se exploró esta columna por separado y no se identificaron hallazgos importantes. Una ve generado el reporte se tiene los siguientes hallazgos:


*   La columna de 'type' está altamente desbalanceada, con menos del 1% de valores distintos
*   Se encontraron varias columnas que cuentan con un porcentaje mayor al 60% de nulls, lo cual implica que la data no es util para analisis y pueden ser eliminadas del data frame.
* Los datos no contienen columnas duplicadas

In [ ]:
profile = ProfileReport(df.drop(columns=['url']), title="Series enero 2024")
profile.to_file("profiling_series_2024.html")
profile.to_notebook_iframe()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

5. De acuerdo al profiling del punto anterior, realizar operaciones de limpieza a los
datos de los dataframes en caso de ser necesario.


In [ ]:

# Drop columns with more than 70% null values
threshold = 0.7
df = df.dropna(thresh=len(df) * threshold, axis=1)


# Convert data types
df['airdate'] = pd.to_datetime(df['airdate'])
df['airstamp'] = pd.to_datetime(df['airstamp'])
df['airtime'] = pd.to_datetime(df['airtime']).dt.time
df['_embedded.show.premiered'] = pd.to_datetime(df['_embedded.show.premiered'])
df['_embedded.show.schedule.time'] = pd.to_datetime(df['_embedded.show.schedule.time']).dt.time

# Create dataframe with show data

show_columns = [col for col in df.columns if col.startswith('_embedded.show')]
df[['_embedded.show.genres','_embedded.show.schedule.days']] = df[['_embedded.show.genres','_embedded.show.schedule.days']] .astype(str)
show_df = df[show_columns].copy()
show_df = show_df.drop_duplicates()
show_df = show_df.reset_index(drop=True)
df = df.drop(columns= [col for col in show_columns if col!= '_embedded.show.id'])
df = df.drop(columns= ['_links.show.href', '_links.show.name'])

# change names of the new show df
new_columns_shows = {col: col.split('.', 2)[2] if '.' in col else col for col in show_df.columns}
show_df = show_df.rename(columns=new_columns_shows)


# Change names in the episodes table
df = df.rename(columns={
    '_links.self.href': 'link_self_href',
    '_embedded.show.id': 'show_id'})

# Create dataframe for WebChannel
webchannel_columns = [col for col in show_df.columns if col.startswith('webChannel')]
webchannel_df = show_df[webchannel_columns].copy()
webchannel_df = webchannel_df.drop_duplicates()
webchannel_df = webchannel_df.reset_index(drop=True)
show_df = show_df.drop(columns= [col for col in webchannel_columns if col!= 'webChannel.id'])

#change names in the WebChannel table
new_columns_wc = {col: col.split('.', 1)[1] if '.' in col else col for col in webchannel_df.columns}
webchannel_df = webchannel_df.rename(columns=new_columns_wc)



print(df.head())

6. Almacenar los DataFrames en archivos parquet (con compresión snappy).


In [ ]:
if not os.path.exists('data'):
    os.makedirs('data')

df.to_parquet('data/episodes.parquet', compression='snappy')
show_df.to_parquet('data/shows.parquet', compression='snappy')
webchannel_df.to_parquet('data/webchannel.parquet', compression='snappy')

7. Leer los archivos parquet del punto anterior y almacenar esta información en una
base de datos (sugerimos sqlite), en un modelo de datos definido por ustedes que
respete la integridad de los datos.


In [ ]:
def read_parquet_files_and_store_in_sqlite(parquet_files_path, db_name='pruebade.db'):
    """Reads parquet files from a directory and stores them in a SQLite database."""

    try:
        # Connect to the SQLite database (creates it if it doesn't exist)
        conn = sqlite3.connect(db_name)

        # Find all parquet files in the specified directory
        parquet_files = glob.glob(os.path.join(parquet_files_path, '*.parquet'))

        if not parquet_files:
          print(f"No parquet files found in {parquet_files_path}")
          return

        # Read and append each parquet file to the SQLite database
        for parquet_file in parquet_files:
            try:
              table_name = os.path.splitext(os.path.basename(parquet_file))[0]
              df = pd.read_parquet(parquet_file)
              df.to_sql(table_name, conn, if_exists='append', index=False)
              print(f"Data from '{parquet_file}' appended to the '{table_name}' table.")

            except Exception as e:
              print(f"Error processing '{parquet_file}': {e}")

        # Commit the changes and close the connection
        conn.commit()
        conn.close()

        print(f"Successfully stored parquet files in '{table_name}' table of '{db_name}'.")

    except Exception as e:
        print(f"An error occurred: {e}")


# Example Usage (replace with your actual paths)
parquet_files_path = 'data'  # Replace with the actual directory
read_parquet_files_and_store_in_sqlite(parquet_files_path)

Data from 'data/episodes.parquet' appended to the 'episodes' table.
Data from 'data/shows.parquet' appended to the 'shows' table.
Data from 'data/webchannel.parquet' appended to the 'webchannel' table.
Successfully stored parquet files in 'webchannel' table of 'pruebade.db'.


8. A partir de los archivos parquets o de la base de datos, realizar operaciones de
agregación para obtener para todos los shows del mes:
a. Runtime promedio (averageRuntime).
b. Conteo de shows de tv por género.
c. Listar los dominios únicos (web) del sitio oficial de los shows.

In [ ]:
def get_average_runtime(db_name='pruebade.db'):
    """Calculates the average runtime per show from the SQLite database."""
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the SQL query
        cursor.execute("""
            SELECT s.name, AVG(e.runtime)
            FROM episodes AS e
            LEFT JOIN shows AS s ON e.show_id = s.id
            WHERE e.runtime IS NOT NULL
            GROUP BY e.show_id, S.name;
        """)

        results = cursor.fetchall()

        # Print the results or process as needed
        for show_name, avg_runtime in results:
            print(f"Show: {show_name}, Average Runtime: {avg_runtime}")
        #
        conn.close()

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")



# Call the function to get the average runtime
get_average_runtime()

Show: Bad Education, Average Runtime: 29.0
Show: QI, Average Runtime: 29.0
Show: The Tonight Show Starring Jimmy Fallon, Average Runtime: 60.0
Show: Casualty, Average Runtime: 49.75
Show: EastEnders, Average Runtime: 29.0
Show: WWE Monday Night RAW, Average Runtime: 180.0
Show: Days of Our Lives, Average Runtime: 39.0
Show: Question Time, Average Runtime: 58.333333333333336
Show: Waterloo Road, Average Runtime: 57.875
Show: Silent Witness, Average Runtime: 58.5
Show: Mrs. Brown's Boys, Average Runtime: 29.0
Show: UFC Fight Night, Average Runtime: 120.0
Show: WWE NXT, Average Runtime: 126.2
Show: Goede Tijden, Slechte Tijden, Average Runtime: 23.0
Show: WWE Premium Live Events, Average Runtime: 240.0
Show: Father Brown, Average Runtime: 44.0
Show: Hollyoaks, Average Runtime: 24.2
Show: Neighbours, Average Runtime: 22.94736842105263
Show: Simon's Cat, Average Runtime: 2.0
Show: Rooster Teeth Podcast, Average Runtime: 90.0
Show: Infoman, Average Runtime: 30.0
Show: NFL Films Presents, Ave

In [ ]:
def get_dataframe_from_sql(db_name, query):

    try:
        conn = sqlite3.connect(db_name)
        df = pd.read_sql_query(query, conn)
        conn.close()
        return df
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        return None


db_name = 'pruebade.db'
query = "SELECT name, genres FROM shows;"

df_shows_genres = get_dataframe_from_sql(db_name, query)
df_shows_genres['genres'] = df_shows_genres['genres'].apply(eval)
genre_counts = df_shows_genres['genres'].explode().value_counts()
print(genre_counts)


genres
Drama              154
Comedy             119
Romance             72
Adventure           70
Fantasy             69
Action              59
Crime               44
Anime               41
Mystery             29
Thriller            28
History             24
Children            23
Food                22
Sports              19
Travel              18
Music               16
Family              15
War                 12
Science-Fiction     12
Supernatural         8
Medical              8
Nature               7
Horror               7
DIY                  7
Legal                4
Adult                2
Name: count, dtype: int64


In [ ]:
def get_unique_domains(db_name='pruebade.db'):
    try:
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the SQL query
        cursor.execute("""
            SELECT DISTINCT name, officialSite
            FROM shows s
            WHERE officialSite IS NOT NULL;
        """)
        results = cursor.fetchall()

        # Print the results or process as needed
        for show_name, officialSite in results:
            print(f"Show: {show_name}, officialSite: {officialSite}")
        #
        conn.close()

    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

get_unique_domains()

Show: вМесте. Интервью, officialSite: http://vmesteproject.ru/intervue/
Show: КСТАТИ, officialSite: https://vk.com/video/playlist/-220754053_3
Show: Wu Dong Qian Kun, officialSite: https://v.qq.com/x/search/?q=%E6%AD%A6%E5%8A%A8%E4%B9%BE%E5%9D%A4&stag=&smartbox_ab=
Show: Wan Jie Du Zun, officialSite: https://v.qq.com/x/cover/mzc00200cu8uq8c.html
Show: Wu Ying Sanqian Dao, officialSite: https://v.qq.com/x/cover/mzc0020097rnzcv.html
Show: The Magic Chef of Ice and Fire, officialSite: https://so.youku.com/search_video/q_冰火魔厨?spm=a2hbt.13141534.left-title-content-wrap.5~A
Show: Lian Qi Shi Wan Nian, officialSite: https://v.qq.com/x/cover/mzc002006n62s11.html
Show: Indulgence, officialSite: https://v.youku.com/v_nextstage/id_bddb8bd0925845bcb56e.html?spm=a2h0c.8166622.PhoneSokuProgram_1.dtitle
Show: Fanren Xiu Xian Chuan Zhi Fanren Feng Qi Tian Nan, officialSite: https://www.bilibili.com/bangumi/media/md28223043/
Show: Tokyo Joshi Pro Wrestling, officialSite: https://www.ddtpro.com/
Show: K